In [2]:
with open(r"C:\Users\shekh\Desktop\LLM from scratch\LLM-from-scratch\data\the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [3]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


<div class="alert alert-block alert-info">
The result is a list of individual words, whitespaces, and punctuation characters:
</div>


<div class="alert alert-block alert-warning">

Let's modify the regular expression splits on whitespaces (\s) and commas, and periods
([,.]):</div>

In [4]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


<div class="alert alert-block alert-info">
We can see that the words and punctuation characters are now separate list entries just as
we wanted
</div>


<div class="alert alert-block alert-warning">

A small remaining issue is that the list still includes whitespace characters. Optionally, we
can remove these redundant characters safely as follows:</div>

In [5]:
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


REMOVING WHITESPACES OR NOT

When developing a simple tokenizer, whether we should encode whitespaces as
separate characters or just remove them depends on our application and its
requirements. Removing whitespaces reduces the memory and computing
requirements. However, keeping whitespaces can be useful if we train models that
are sensitive to the exact structure of the text (for example, Python code, which is
sensitive to indentation and spacing). Here, we remove whitespaces for simplicity
and brevity of the tokenized outputs. Later, we will switch to a tokenization scheme
that includes whitespaces.



The tokenization scheme we devised above works well on the simple sample text. Let's
modify it a bit further so that it can also handle other types of punctuation, such as
question marks, quotation marks, and the double-dashes we have seen earlier in the first
100 characters of Edith Wharton's short story, along with additional special characters: 

In [6]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [7]:
# Strip whitespace from each item and then filter out any empty strings.
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [8]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [9]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [10]:
print(len(preprocessed))


4690


In [11]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [12]:
vocab = {token:integer for integer,token in enumerate(all_words)}


In [13]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [14]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [15]:
tokenizer = SimpleTokenizerV1(vocab)
text = """It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [16]:
tokenizer.decode(ids)

'It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [17]:
text = "Hello, do you like tea?"
print(tokenizer.encode(text)) 

KeyError: 'Hello'

The problem is that the word hello was not used in the the verdict story.
hence it is not contained in the vocabulary
this highlights the need to consider large and diverse training sets to extend the vocabulary when working on llm 

Adding Special conetxt tokens

in the previous seciton, we implemented s smimple tokenizer and applied it to a a passage form the training set.
in this section we will modify this tokenizer to handle unknown words
in patrticular we will modify the viocabulary and tokenizer we implemented in the previous section, simple tokenizerv2, to support two new tokens , <|unk|> and <|endoftext|>


We can modify the tokenizer to use an <|unk|> token if it
encounters a word that is not part of the vocabulary. 

Furthermore, we add a token between
unrelated texts. 

For example, when training GPT-like LLMs on multiple independent
documents or books, it is common to insert a token before each document or book that
follows a previous text source

Let's now modify the vocabulary to include these two special tokens, <unk> and
<|endoftext|>, by adding these to the list of all unique words that we created in the
previous section:

In [18]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>","<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [19]:
len(vocab.items())

1132

In [20]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [21]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [22]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

text

'Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.'

In [23]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [24]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

So far, we have discussed tokenization as an essential step in processing text as input to
LLMs. Depending on the LLM, some researchers also consider additional special tokens such
as the following:

[BOS] (beginning of sequence): This token marks the start of a text. It
signifies to the LLM where a piece of content begins.

[EOS] (end of sequence): This token is positioned at the end of a text,
and is especially useful when concatenating multiple unrelated texts,
similar to <|endoftext|>. For instance, when combining two different
Wikipedia articles or books, the [EOS] token indicates where one article
ends and the next one begins.

[PAD] (padding): When training LLMs with batch sizes larger than one,
the batch might contain texts of varying lengths. To ensure all texts have
the same length, the shorter texts are extended or "padded" using the
[PAD] token, up to the length of the longest text in the batch.


Note that the tokenizer used for GPT models does not need any of these tokens mentioned
above but only uses an <|endoftext|> token for simplicity


the tokenizer used for GPT models also doesn't use an <|unk|> token for outof-vocabulary words. Instead, GPT models use a byte pair encoding tokenizer, which breaks
down words into subword units

## BYTE PAIR ENCODING

since implementing BPE can be complicated we will use an existing Python open source library called tiktoken.
this library implements the BPE algorithm very efficiently based on the source code in rust

In [25]:
# !pip3 install tiktoken

In [26]:
import importlib
import tiktoken

print("tiktoken verison: ", importlib.metadata.version("tiktoken"))

tiktoken verison:  0.8.0


once installed we can instantiate the bpe tokenizer form the tiktoken as follows:


In [27]:
tokenizer = tiktoken.get_encoding('gpt2')

the usage of this tokenizer is similar to simpletokenizerv2 we implement previoulsy via an encode method

In [28]:
text = ("Hello, do you like tea? <|endoftext|> In the sunlit terraces"
        "of sumeunknownPlace.")

inetgers = tokenizer.encode(text,allowed_special={"<|endoftext|>"})

print(inetgers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 2160, 68, 34680, 27271, 13]


we can now convert the token IDs back to text using decode method, similar to our simpletokenizerv2 decode method 


In [29]:
strings = tokenizer.decode(inetgers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof sumeunknownPlace.


we can make 2 noteworthy observations based on the token IDs and decoded text

first, the <|endoftext|> token is assigned a relatively large token ID namely 50256

in fact, the BPE tokenizer which was used to train the ,models GPT-2, GPT-3 and the original model used in chatgpt has a total vocab size of 50257 with <|endoftext|> assigned the largest token ID


second the BOE tokenizer above encodes and decodes unknow words such "someunknowplace" correctly

BPW tokenizer can handle any unknown word how does it achieve this without using <|unk|> tokens?



the algorithm underlying BPE breaks down words that arent its predefined vocabulary into smaller subword units or even individual characters.

this enables it to handle out of vocabulary words

so if the tokenizer encounters an unfamiliar word during tokenization, it can represent it as a sequence of subwords tokens or characters.

In [30]:
# last example to illustrate how BOE tokinizer deals with unkown words.

integers = tokenizer.encode("Akwirw ier")
print(integers)

strings = tokenizer.decode(integers)
print(strings)

[33901, 86, 343, 86, 220, 959]
Akwirw ier


# Creating INPUT-TARGET pairs -lecture 9

<div class="alert alert-block alert-success">
  In this section we implement a data loader that fetches the input-target pairs using a sliding window approach 

</div>


<div class="alert alert-block alert-success">
  to get started, we will first tokenize the whole The Verdict story we worked with earlier using the BPE(Byte pair encoding) tokenizer introduced previously. 

</div>


In [33]:
with open(r"data\the-verdict.txt",'r',encoding='utf-8') as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


<div class="alert alert-block alert-info">
  executing the above code will return 5145, the total number of tokens in the training set, after applying the BPE tokenizer

</div>


<div class="alert alert-block alert-success">
  Next we remove the first 50 tokens from the dataset for demonstration purposes as it results in a slightly more interesting test passage in the next step:

</div>


In [35]:
enc_sample = enc_text[50:]

<div class="alert alert-block alert-success">
  One of the easiest and most intuitive ways to create input-target pairs for the next word prediction is to create two variable x and y where x contains the input token and y contains the targets, which are inputs shifted by 1.

</div>


<div class="alert alert-block alert-info">
  The conetxt size determines how many tokens are included in the input

</div>


In [39]:
conetxt_size = 4 #length of the input
# the context size of 4 means the is trained to look at a sequence of 4 words(or tokens)
# to predict the next word in the sequence
# the first 4 tokens [1,2,3,4] and the target y is the next 4 tokens [2,3,4,5]

x = enc_sample[:conetxt_size]
y = enc_sample[1:conetxt_size+1]

print("x:",x)   

print("y:  ",y)


x: [290, 4920, 2241, 287]
y:   [4920, 2241, 287, 257]


In [42]:
for i in range(1,conetxt_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(f"context: {context},-----> desired: {desired}")

context: [290],-----> desired: 4920
context: [290, 4920],-----> desired: 2241
context: [290, 4920, 2241],-----> desired: 287
context: [290, 4920, 2241, 287],-----> desired: 257


<div class="alert alert-block alert-info">
  everything left of the arrow is refers to the input llm will receive and the token id on the right side of the arrow is what llm  is supposed to predict

  now for lets convert the token id into text
</div>


In [46]:
for i in range(1,conetxt_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->",tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


<div class="alert alert-block alert-info">
  we have now created the input target pairs that we can turn into use for the llm training in upcoming chapters
</div>


<div class="alert alert-block alert-info">
  there's only one more task before we can turn the tokens into embeddings: implementing an efficient data loader that iterates throught the input dataset and returns the inputs and targets as pytorch tensors, which can be thought of as multidimensional arrays
</div>


<div class="alert alert-block alert-info">
  In particular, we are interested in returning two tensors: an input tensor containing the text that LLM sees and target tensor that includes target for the LLM to predict
</div>


# Implementing Data Loader

<div class="alert alert-block alert-success">
for the efficient data loader implementation we will use pytorch's built in dataset and data loader classes
</div>


<div class="alert alert-block alert-success">
Step1: tokenize the entire text  <br>
step2: use a sliding window to chunk the book into overlapping sequences of the max_length  <br>
step3: return the total number of rows in the dataset  <br>
step4: return a single row from the dataset  <br>
</div>
